## Working with the pyapacheatlas Python Library

##### The goal of this notebook is to demonstrate a programmatic way to update Azure SQL table column descriptions.

[https://github.com/wjohnson/pyapacheatlas](https://github.com/wjohnson/pyapacheatlas)

#### Setup

In [105]:
%pip install pyapacheatlas

StatementMeta(, , -1, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [106]:
import json
import os
import openai
import requests
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient, AtlasClassification, AtlasEntity, AtlasProcess
from pyapacheatlas.core.util import AtlasException
import pyodbc
import uuid
from pyspark.sql.functions import col

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 197, Finished, Available)

#### Get secrets from Key Vault
- Provide the name of your keyvault.
- There are many secrets fetched in this notebook.
- You may choose to statically assign these values instead of using Key Vault.

In [ ]:
kv_name = ''

# Uncomment the below to statically assign the required variables.
# sql_uname = ''
# sql_pass = ''
# tenant_id = ''
# client_id = ''
# client_secret = ''
# sql_uri = ''
# purview_name = ''

In [107]:
from trident_token_library_wrapper \
import PyTridentTokenLibrary as tl

access_token = mssparkutils.credentials.getToken("keyvault")

# Fetch the SQL username used to connect to Azure SQL.
key = 'mcaps-sql-test-username'

sql_uname = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the SQL user password to connecto to Azure SQL.
key = 'mcaps-sql-test-pass'

sql_pass = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the Entra ID tenant id.
key = 'mcaps-tenant-id'

tenant_id = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the app registration used to authenticate against Microsoft Purview.
key = 'mcaps-appreg-purview'

client_id = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the app registrated secret.
key = 'mcaps-appreg-purview-key'

client_secret = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the URI of the SQL server instance.
key = 'mcaps-sql-server-uri'

sql_uri = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)

# Fetch the Microsoft Purview account name.
key = 'mcaps-purview-name'

purview_name = tl.get_secret_with_token( \
  f"https://{kv_name}.vault.azure.net/", \
  key, \
  access_token)


StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 198, Finished, Available)

In [108]:
# Authenticate against your Atlas server
oauth = ServicePrincipalAuthentication(
    tenant_id= tenant_id,
    client_id= client_id,
    client_secret= client_secret
)
client = PurviewClient(
    account_name = purview_name,
    authentication=oauth
)

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 199, Finished, Available)

In [119]:
# Check if the table exists in Purview
def check_entity_if_exist(client, qualifiedName, typeName):
    entities = client.get_entity(qualifiedName=qualifiedName, typeName=typeName)
    #return len(entities) > 0
    return entities

# Update column description in Purview
def update_entity(client, guid, userDescription):
    client.partial_update_entity(
    guid=guid,
    attributes={
        "userDescription" : userDescription
    }
)

# Get the value of the Description extended property for a column.
def get_description(df, column_name):
    epn = "Description"
    pdf = df.where(col("ColumnName") == column_name).where(col("ExtendedPropertyName") == epn).select("ExtendedPropertyValue").toPandas()
    dict = pdf.to_dict(orient = 'list')
    dict_length = len(dict['ExtendedPropertyValue'])
    if dict_length > 0:
        return dict['ExtendedPropertyValue'][0]

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 210, Finished, Available)

### Get extended properties from Azure SQL

In [110]:
jdbcHostname = sql_uri

# My example database is called "sql-test"
jdbcDatabase = "sql-test"


jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

view_name = "OrdersProperties"

query = f"(SELECT SchemaName, TableName, ColumnName, ExtendedPropertyName, cast(ExtendedPropertyValue as varchar(255)) as ExtendedPropertyValue FROM {view_name}) as view_query"

connectionProperties = {
"user" : sql_uname,
"password" : sql_pass,
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 201, Finished, Available)

In [111]:
# sql-test is serverless and may fail on the first attempt

Spdf = spark.read.jdbc(
    url=jdbcUrl,
    table=query,
    properties=connectionProperties
)

display(Spdf)

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 202, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5a0ddb79-4f98-49cd-8cc7-8db20698d31f)

### Add Column Description

There are 3 properties for description: sqlDescription, userDescription, and Description. <mark><u>**userDescription**</u></mark> is the appropriate property to update.

example:

```
"userDescription": "Description of order id.",
```

In [112]:
sql_db = "sql-test"
sql_schema = "dbo"
sql_table = "Orders"

entity = check_entity_if_exist(client, "mssql://{0}/{1}/{2}/{3}".format(sql_uri, sql_db, sql_schema, sql_table), "azure_sql_table")

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 203, Finished, Available)

In [113]:
print(entity)

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 204, Finished, Available)

{'referredEntities': {'fa2db9ec-b07e-4200-9d7d-30f6f6f60003': {'typeName': 'azure_sql_column', 'attributes': {'owner': None, 'userTypeId': 56, 'columnEncryptionKeyDatabaseName': None, 'replicatedTo': None, 'userDescription': None, 'replicatedFrom': None, 'qualifiedName': 'mssql://[REDACTED]/sql-test/dbo/Orders#ProductId', 'displayName': None, 'sqlDescription': None, 'precision': 10, 'length': 4, 'encryptionType': 0, 'columnEncryptionKeyId': 0, 'description': None, 'scale': 0, 'isXmlDocument': 'false', 'isMasked': 'false', 'encryptionTypeDesc': None, 'xmlCollectionId': 0, 'isHidden': 'false', 'name': 'ProductId', 'data_type': 'int', 'encryptionAlgorithmName': None, 'systemTypeId': 56}, 'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60003', 'isIncomplete': False, 'provenanceType': 0, 'status': 'ACTIVE', 'createdBy': 'ServiceAdmin', 'updatedBy': 'ServiceAdmin', 'createTime': 1706020729998, 'updateTime': 1706020729998, 'version': 0, 'isIndexed': True, 'source': 'DataScan', 'sourceDetails': {'Scan

Get a list of column guids and update the **userDescription** for each. Columns exist in the top level "entities" within the object "relationshipAttributes".

In [114]:
columns = entity["entities"][0]["relationshipAttributes"]["columns"]
print(columns)

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 205, Finished, Available)

[{'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60002', 'typeName': 'azure_sql_column', 'entityStatus': 'ACTIVE', 'displayText': 'OrderId', 'relationshipType': 'azure_sql_table_columns', 'relationshipGuid': '0abb7d89-7fc3-49aa-8426-a311b0379aa4', 'relationshipStatus': 'ACTIVE', 'relationshipAttributes': {'typeName': 'azure_sql_table_columns'}}, {'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60003', 'typeName': 'azure_sql_column', 'entityStatus': 'ACTIVE', 'displayText': 'ProductId', 'relationshipType': 'azure_sql_table_columns', 'relationshipGuid': 'adba3941-f90c-4fd6-9f07-57abf929e8e4', 'relationshipStatus': 'ACTIVE', 'relationshipAttributes': {'typeName': 'azure_sql_table_columns'}}, {'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60001', 'typeName': 'azure_sql_column', 'entityStatus': 'ACTIVE', 'displayText': 'Comments', 'relationshipType': 'azure_sql_table_columns', 'relationshipGuid': '1d0803d1-c042-4127-9714-b72813524d84', 'relationshipStatus': 'ACTIVE', 'relationshipAttributes': {'typeName': 'azur

In [115]:
columns_list = []

for i in columns:
    dict = {
        "name" : i['displayText'],
        "guid" : i['guid']
    }
    columns_list.append(dict)

print(columns_list)

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 206, Finished, Available)

[{'name': 'OrderId', 'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60002'}, {'name': 'ProductId', 'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60003'}, {'name': 'Comments', 'guid': 'fa2db9ec-b07e-4200-9d7d-30f6f6f60001'}]


In [120]:
for element in columns_list:
    name = element['name']
    userDescription = get_description(Spdf, element['name'])
    if userDescription:
        update_entity(client, element['guid'], userDescription)
        print("Column {0} updated with description: {1}.".format(name, userDescription ))

StatementMeta(, ec1f2748-1503-40a0-b0d8-a24a08d92785, 211, Finished, Available)

Column OrderId updated with description: Primary key for orders. [REDACTED] 2/5/2024.
Column Comments updated with description: String based text collected from order form..
